## Calculate the chance of being born in Canada as a function of the year

### First, I'll pull the data from the Worldbank API
### Then I'll store the data in a sqlite database
### Next, I'll query the database to get the data I need into a pandas dataframe

### To calculate the chance of being born in Canada as a function of the year, I will calculate the raw births per year for Canada and the world.

### -      births per year = (birth rate / 1000) * 1000 * population

### Then I'll divide the number of births in Canada by the number of births in the world to get the chance of being born in Canada as a function of the year.


In [1]:
import matplotlib.pylab as plt
import pandas as pd
import sqlite3
import requests
import time
from datetime import datetime

In [2]:
birth_rate_source = 'https://api.worldbank.org/v2/country/all/indicator/SP.DYN.CBRT.IN'
population_source = 'https://api.worldbank.org/v2/country/all/indicator/SP.POP.TOTL'

# "CA" Canada country code

params = {
    'format': 'json',
    'per_page': 1000,  # Maximum records per page
    'page': 1,
    'date': '1973:2023',
}
db_path = "public_data.db"

In [3]:
"""Create SQLite database and tables"""
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Create a general table for World Bank indicators
cursor.execute("""
    CREATE TABLE IF NOT EXISTS world_bank_data (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        country_code TEXT,
        country_name TEXT,
        indicator_code TEXT,
        indicator_name TEXT,
        year INTEGER,
        value REAL,
        decimal_places INTEGER,
        obs_status TEXT,
        unit TEXT,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
""")

# Create an index for faster queries
cursor.execute("""
    CREATE INDEX IF NOT EXISTS idx_country_indicator_year 
    ON world_bank_data(country_code, indicator_code, year)
""")

conn.commit()
conn.close()
print(f"Database created: {db_path}")

Database created: public_data.db


In [4]:
def pull_wb_data(source_url):
    """Pull data from the source worldbank URL."""
    all_data = []
    session = requests.Session()
    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (compatible; DataFetcher/1.0)'
    })

    while True:
        response = session.get(source_url, params=params, timeout=30)
        response.raise_for_status()

        data = response.json()
        
        # World Bank API returns metadata in first element, data in second
        if len(data) < 2 or not data[1]:
            break
        
        metadata = data[0]
        records = data[1]
        
        all_data.extend(records)
        
        # Check if there are more pages
        if params['page'] >= metadata.get('pages', 1):
            break
        
        params['page'] += 1
        time.sleep(0.5)

    print(f"Successfully fetched {len(all_data)} records")
    params['page'] = 1  # Reset page index

    return all_data

In [5]:
birth_rate_data = pull_wb_data('https://api.worldbank.org/v2/country/all/indicator/SP.DYN.CBRT.IN')
population_data = pull_wb_data('https://api.worldbank.org/v2/country/all/indicator/SP.POP.TOTL')

Successfully fetched 13566 records
Successfully fetched 13566 records


In [6]:
def store_data(data, indicator):
    """Save World Bank data to SQLite database"""
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    cursor.execute("DELETE FROM world_bank_data WHERE indicator_code = ?", (indicator,))

    # Prepare data for insertion
    records = []
    for item in data:
        if item is None:
            continue
            
        record = (
            item.get('country', {}).get('id', ''),
            item.get('country', {}).get('value', ''),
            item.get('indicator', {}).get('id', ''),
            item.get('indicator', {}).get('value', ''),
            item.get('date'),
            item.get('value'),
            item.get('decimal'),
            item.get('obs_status'),
            item.get('unit'),
        )
        records.append(record)

    # Insert data
    cursor.executemany("""
        INSERT INTO world_bank_data 
        (country_code, country_name, indicator_code, indicator_name, year, value, 
            decimal_places, obs_status, unit) 
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, records)

    conn.commit()
    conn.close()
    print(f"Saved {len(records)} records to database")

In [7]:
store_data(birth_rate_data, "birthrate")
store_data(population_data, "population")

Saved 13566 records to database
Saved 13566 records to database


In [8]:
def query_data(query: str):
    """Execute a query on the database"""
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    cursor.execute(query)
    results = cursor.fetchall()
    results = pd.DataFrame(results)
    
    conn.close()
    return results

In [9]:
# SP.POP.TOTL = population
# SP.DYN.CBRT.IN = birthrate

In [13]:
query = """
        SELECT year, country_code, value
        FROM world_bank_data
        WHERE indicator_code='SP.DYN.CBRT.IN'
    """
birth_rate_data = query_data(query=query)
print(birth_rate_data)  # Quick test

          0   1          2
0      2023  ZH  32.954912
1      2022  ZH  33.320433
2      2021  ZH  33.681336
3      2020  ZH  34.010470
4      2019  ZH  34.335881
...     ...  ..        ...
13561  1977  ZW  46.510000
13562  1976  ZW  46.524000
13563  1975  ZW  46.623000
13564  1974  ZW  46.395000
13565  1973  ZW  46.384000

[13566 rows x 3 columns]


In [14]:
query = """
        SELECT year, country_code, value
        FROM world_bank_data
        WHERE indicator_code='SP.POP.TOTL'
    """
population_data = query_data(query=query)
print(population_data)  # Quick test

          0   1            2
0      2023  ZH  750503764.0
1      2022  ZH  731821393.0
2      2021  ZH  713090928.0
3      2020  ZH  694446100.0
4      2019  ZH  675950189.0
...     ...  ..          ...
13561  1977  ZW    6449512.0
13562  1976  ZW    6287105.0
13563  1975  ZW    6098658.0
13564  1974  ZW    5908328.0
13565  1973  ZW    5720404.0

[13566 rows x 3 columns]


In [20]:
combined_dataframe = pd.merge(left=population_data, right=birth_rate_data, how='left', left_on=[0, 1], right_on=[0, 1])
combined_dataframe = combined_dataframe.rename(columns={0: "Year", 1:"Country Code", "2_x": "Population", "2_y": "Birth rate / 1000",})
combined_dataframe

,Year,Country Code,Population,Birth rate / 1000
0,2023,ZH,750503764.0,32.954912
1,2022,ZH,731821393.0,33.320433
2,2021,ZH,713090928.0,33.681336
3,2020,ZH,694446100.0,34.010470
4,2019,ZH,675950189.0,34.335881
...,...,...,...,...
13561,1977,ZW,6449512.0,46.510000
13562,1976,ZW,6287105.0,46.524000
13563,1975,ZW,6098658.0,46.623000
13564,1974,ZW,5908328.0,46.395000
